# Task 2    
results: g1, g7, g2, g8

In [3]:
import pygad 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def read_graph(path):
    with open(path) as f: n = int(f.readline())
    print(n)
    graph = pd.read_csv(path, sep=" ", skiprows=1, names=["A", "B", "w"])
    print(graph)
    return n, graph

In [51]:
# choose graph
path = "../graphs/g8.txt"

n, graph = read_graph(path)
k = 2    # genome length scaling factor

# select a sequence to be visited
sequence = [1,15]

15
     A   B  w
0    1   3  5
1    1   6  9
2    1   8  4
3    2   4  3
4    2   6  7
5    2   7  8
6    2   9  4
7    3   7  3
8    3   8  5
9    4   5  2
10   4   8  5
11   4   9  1
12   4  13  6
13   5   7  2
14   5   8  3
15   6   7  7
16   7  15  1
17   6   9  4
18   8   9  9
19   8  14  2
20   9  10  4
21   9  11  2
22  10  12  3
23  11   1  1
24  12   3  4
25  12  13  4
26  13   2  2
27  13   5  7
28  14  11  1
29  15   1  7


In [45]:
# fitness function
def pathLength_multipleNodes(pygad_instance, path, solution_index):
    INVALID = -1e12
    it = 0
    dist = 0
    prev = 0

    for i in range(len(path)-1): 
        node1, node2 = path[i], path[i+1]
        if(node1 == node2 or node2 == 0): 
            prev = node1
            continue
        if(node1 == 0): node1 = prev

        if(node1 == sequence[it]): it += 1
        if(it == len(sequence)): return -dist

        w = graph.loc[(graph['A'] == node1) & (graph['B'] == node2), 'w']
        if w.empty:
            return INVALID
        
        dist += w.values[0]

    if(path[-1] == sequence[it]): return -dist
    return INVALID


In [27]:
# intial population with constraints
def creating_init_pop(solutions_per_pop):
    genom_len = k*n
    pop = []

    for _ in range(solutions_per_pop):   # v oklepaju solutions per population
        # all the nodes we can add
        full_gene_space = [node for node in range(n+1)]
        number_of_nodes_to_insert = genom_len - len(sequence)      # je odvisno od num_genes

        # we decide on which indices we will insert random nodes into original sequence
        # we cannot insert them in [0] and [n-1] position (start and end)

        # list of indices on which we are going to insert new nodes
        positions = []

        # list of indices we can choose from
        possible_indices = list(range(genom_len))

        for _ in range(number_of_nodes_to_insert):
            # random choice of possible indices
            idx = np.random.choice(possible_indices)
            # we remove it from options
            possible_indices.remove(idx)
            # add to positions
            positions.append(idx)
        
        # all thats left is to build our startion population 
        # in indices from position we put a random node from full gene space

        # building process :)
        pop_individual = [-1] * genom_len   # list of n elements with value -1

        for x in positions:
            pop_individual[x] = np.random.choice(full_gene_space)
        
        it = 0
        for i in range(genom_len):
            if pop_individual[i] == -1:
                pop_individual[i] = sequence[it]
                it += 1

        # moja logika pravi, da bi moglu bit ok
        pop_individual = np.array(pop_individual)
        pop.append(pop_individual)
    
    #print(pop)
    return pop


# crossover function with those constraints
def custom_crossover(parents, offspring_size, ga_instance):
    offspring = []

    # offspring_size is a touple of 2 numbers: (the offspring size, number of genes)
    size_of_offspring, number_of_genes = offspring_size

    parents_size = len(parents)

    def check_sequence(p1, p2, idx):
        id = 0
        for i in range(idx):
            if p1[i]==sequence[id]:
                id += 1
            if id == len(sequence):
                return True
        for i in range(idx, number_of_genes):
            if p2[i]==sequence[id]:
                id += 1
            if id == len(sequence):
                return True

        return False

    for i in range(size_of_offspring):
        # randomly choose 2 parents
        parent_indices = np.random.choice(parents_size, 2, replace=False)
        parent1 = parents[parent_indices[0]]
        parent2 = parents[parent_indices[1]]

        # now we have two random parents we chose for mating
    
        # idea: we traverse both parents and save possible points for crossovers
        # then we randomly select a point for crossover from the available points

        available_points_for_crossover = []

        # iterating over sequence

        for pos in range(0, number_of_genes):
            if check_sequence(parent1, parent2, pos):
                available_points_for_crossover.append(pos)
        
        chosen_crossover_point = np.random.choice(available_points_for_crossover)

        # combine parents to get child
        individual_offspring = np.concatenate([parent1[:chosen_crossover_point], parent2[chosen_crossover_point:]])
        offspring.append(individual_offspring)

    return np.array(offspring)


# mutation function with the same constraints
def custom_mutation(offspring, ga_instance):
    # after crossover, the offspring is handed to mutation
    # idea: swap two genes - not if they are both in sequence

    for offspring_individual in offspring:
        length = len(offspring_individual)
        swap_points = []
        it = 0

        for i in range(0, length-1):
            if offspring_individual[i] not in sequence:
                swap_points.append(i)
            if it < len(sequence)-1 and offspring_individual[i] == sequence[it]:
                it += 1       

        if swap_points:
            chosen_swap_point = np.random.choice(swap_points)
            offspring_individual[chosen_swap_point], offspring_individual[chosen_swap_point+1] = offspring_individual[chosen_swap_point+1], offspring_individual[chosen_swap_point]      

    return offspring


In [32]:
def genetic_algo_task2(number_generations, number_parents_mating, solutions_per_pop):
    init_pop = creating_init_pop(solutions_per_pop)
    gene_space = [i for i in range(n+1)]


    ga_instance = pygad.GA(fitness_func=pathLength_multipleNodes,
                       num_generations=number_generations,
                       num_parents_mating=number_parents_mating,
                       initial_population=init_pop,
                       sol_per_pop=solutions_per_pop,
                       num_genes=(n*k),
                       gene_space=gene_space,
                       gene_type=int,
                       crossover_type=custom_crossover,
                       mutation_type=custom_mutation,
                       mutation_probability=0.2
                      )

    ga_instance.run()

    #solution, solution_fitness, solution_idx = ga_instance.best_solution()
    #print(-solution_fitness)

    print("best solution: ", ga_instance.best_solution())
    #ga_instance.plot_fitness()

    return abs(ga_instance.best_solution()[1])


In [53]:
initPop = [10, 25, 50, 100, 200]
numGens = [20, 50, 100, 200, 500]
csv_path = "GenerationsResults.csv"
results = {}

for g in numGens:
    fitness = genetic_algo_task2(
        number_generations=g,
        number_parents_mating=5,
        solutions_per_pop=50
    )

    results[f"gen_{g}"] = fitness  #a map

# change to pandas dataframe
df_row = pd.DataFrame([results])

# Save or append
if not os.path.exists(csv_path):
    df_row.to_csv(csv_path, index=False)
else:
    df_row.to_csv(csv_path, mode="a", header=False, index=False)

best solution:  (array([ 1, 10, 11,  0,  0,  4,  4,  1,  1,  8, 11,  3,  5, 10,  2, 12, 10,
        2, 11,  1,  3,  6,  9,  3, 15,  5,  7,  7,  1,  1]), np.float64(-1000000000000.0), np.int64(0))
best solution:  (array([ 2,  7,  2,  9,  7,  1,  9,  2,  8,  2,  8,  2,  6,  6,  6,  4,  4,
       15,  7, 12, 15, 13, 15, 15,  3,  8,  5,  5,  8,  5]), np.float64(-1000000000000.0), np.int64(0))
best solution:  (array([15,  1, 11,  5,  4,  8,  8,  3,  8, 10,  8,  2, 15,  2, 10, 11, 10,
       15, 13, 15, 13,  2,  1,  1, 15,  0,  0,  5,  5,  5]), np.float64(-1000000000000.0), np.int64(0))
best solution:  (array([15, 13,  1, 13, 13,  4,  4, 15,  5,  4,  5, 15,  4,  8, 15, 15,  5,
        8,  6, 15, 15,  6,  6,  1,  6,  4, 12, 12,  4,  4]), np.float64(-1000000000000.0), np.int64(0))
best solution:  (array([ 1, 15,  4,  4, 15,  9,  9, 15,  4, 15, 15, 15, 15,  9,  9,  1,  9,
        9,  1,  4,  4, 11,  4,  4,  4,  4,  4,  4, 11, 11]), np.float64(-1000000000000.0), np.int64(0))
